In [1]:
#load packages
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import soundfile as sf


In [3]:
#import data 
path = 'x:/Steering/Recordings/Processed/'
data1,_ = sf.read(path + 'ERB=2_MF_01.wav')
data2,_ = sf.read(path + 'ERB=2_MF_02.wav')
data3,_ = sf.read(path + 'ERB=2_MF_03.wav')

classes = pd.read_csv(path + 'Classes_2_MF.csv')

data = np.hstack([data1, data2, data3])

In [4]:
data.shape

(970584, 756)

In [5]:
X = data[:len(data)-1,:data.shape[1]] #775361
Y = classes.values[:,0]

In [6]:
X.shape, Y.shape

((970583, 756), (970583,))

In [7]:
#split into training and test set
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2,random_state=0)

In [8]:
#do one-hot encoding
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder

enc = OneHotEncoder(sparse=False)

In [9]:
integer_encoded_train = trainY.reshape(len(trainY), 1)
onehot_encoded_train = enc.fit_transform(integer_encoded_train)

integer_encoded_test = testY.reshape(len(testY), 1)
onehot_encoded_test = enc.fit_transform(integer_encoded_test)

In [10]:
onehot_encoded_train 


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [11]:
#DC removal 
trainX -= (np.mean(trainX, axis=0) + 1e-8)
testX -= (np.mean(testX, axis=0) + 1e-8)

In [12]:
clf = keras.Sequential()

In [13]:
trainX.shape, onehot_encoded_train.shape

((776466, 756), (776466, 5))

In [14]:
clf.add(keras.layers.Dense(
  units = 380,
  input_dim = trainX.shape[1],   
  activation = 'tanh'
))

clf.add(keras.layers.Dense(
    units = onehot_encoded_train.shape[1],
    input_dim = 380,
    activation = 'softmax'
))


In [15]:
onehot_encoded_train.shape[1]

5

In [16]:
clf.compile(loss='mean_squared_error',
    optimizer='Adam',
           metrics=['categorical_accuracy'])

In [17]:
history = clf.fit(trainX,onehot_encoded_train,
                 batch_size = 64, epochs = 30,
                 verbose = 1,
                 validation_split = 0.1)

Train on 698819 samples, validate on 77647 samples
Epoch 1/30
698819/698819 [==============================] - 188s 269us/step - loss: 0.0452 - categorical_accuracy: 0.8451 - val_loss: 0.0345 - val_categorical_accuracy: 0.8829
Epoch 2/30
698819/698819 [==============================] - 118s 169us/step - loss: 0.0319 - categorical_accuracy: 0.8929 - val_loss: 0.0300 - val_categorical_accuracy: 0.8997
Epoch 3/30
698819/698819 [==============================] - 106s 152us/step - loss: 0.0278 - categorical_accuracy: 0.9075 - val_loss: 0.0265 - val_categorical_accuracy: 0.9123
Epoch 4/30
698819/698819 [==============================] - 105s 150us/step - loss: 0.0250 - categorical_accuracy: 0.9174 - val_loss: 0.0237 - val_categorical_accuracy: 0.9223
Epoch 5/30
698819/698819 [==============================] - 115s 164us/step - loss: 0.0230 - categorical_accuracy: 0.9243 - val_loss: 0.0223 - val_categorical_accuracy: 0.9263
Epoch 6/30
698819/698819 [==============================] - 107s 153u

In [18]:
data.shape[1]

756

In [19]:
clf.predict_classes(testX[1:100,:]) + 1,testY[:100]

(array([2, 5, 1, 5, 2, 4, 4, 2, 4, 4, 3, 2, 3, 1, 3, 3, 2, 4, 3, 4, 2, 4,
        4, 2, 5, 5, 4, 5, 2, 4, 5, 4, 4, 1, 5, 4, 2, 2, 5, 2, 4, 4, 5, 3,
        1, 3, 5, 5, 3, 2, 4, 4, 1, 5, 5, 4, 5, 4, 4, 2, 2, 2, 1, 4, 2, 1,
        2, 2, 1, 4, 2, 1, 4, 2, 1, 5, 3, 4, 5, 4, 3, 5, 2, 5, 4, 2, 2, 2,
        5, 5, 5, 4, 4, 2, 4, 5, 2, 1, 1], dtype=int64),
 array([2, 2, 5, 1, 5, 2, 4, 4, 2, 4, 4, 3, 2, 3, 1, 3, 3, 2, 4, 3, 4, 2,
        4, 4, 2, 5, 5, 4, 5, 2, 4, 5, 4, 4, 1, 5, 4, 2, 2, 5, 2, 4, 4, 5,
        3, 1, 3, 5, 5, 3, 2, 4, 5, 1, 5, 5, 4, 5, 4, 4, 2, 2, 2, 1, 4, 2,
        1, 2, 2, 1, 4, 2, 1, 4, 2, 1, 5, 3, 4, 5, 4, 3, 5, 2, 5, 4, 2, 2,
        2, 4, 5, 5, 4, 4, 2, 4, 5, 2, 1, 1], dtype=int64))

In [20]:
# saving the model 
# save model and weights
clf_json = clf.to_json()
with open("NN_756-380-5.json", "w") as json_file:
    json_file.write(clf_json)
    
clf.save_weights("NN_756-380-5_weights.h5")

In [21]:
np.save('TestX_NN_756-380-5',testX), np.save('TestY_NN_756-380-5',testY),
np.save('OneHot_testY_756-380-5',onehot_encoded_test)

In [22]:
clf.evaluate(testX,onehot_encoded_test)

194117/194117 [==============================] - 10s 50us/step


[0.011338381001418589, 0.9633829082460578]